Load and Preprocess Data

In [ ]:
import os
import cv2
import numpy as np

def load_images_from_folder(folder):
    images = []
    for filename in os.listdir(folder):
        img = cv2.imread(os.path.join(folder, filename))
        if img is not None:
            img = cv2.resize(img, (64, 64))  # Resize to 64x64 for consistency
            images.append(img)
    return np.array(images)

real_images = load_images_from_folder('path_to_real_images')
fake_images = load_images_from_folder('path_to_fake_images')


Generator Model |
The generator creates fake images from random noise.

In [ ]:
from tensorflow.keras.layers import BatchNormalization, ReLU, Dense, Reshape, Conv2DTranspose, LeakyReLU, Dropout, Flatten, Conv2D
from tensorflow.keras.models import Sequential

def build_generator():
    model = Sequential()
    model.add(Dense(256 * 8 * 8, activation="relu", input_dim=100))
    model.add(Reshape((8, 8, 256)))
    model.add(Conv2DTranspose(128, kernel_size=4, strides=2, padding="same"))
    model.add(BatchNormalization(momentum=0.8))
    model.add(ReLU())
    model.add(Conv2DTranspose(64, kernel_size=4, strides=2, padding="same"))
    model.add(BatchNormalization(momentum=0.8))
    model.add(ReLU())
    model.add(Conv2DTranspose(32, kernel_size=4, strides=2, padding="same"))
    model.add(BatchNormalization(momentum=0.8))
    model.add(ReLU())
    model.add(Conv2D(3, kernel_size=3, padding="same", activation='tanh'))
    return model


Discriminator Model | 
The discriminator distinguishes between real and fake images.

In [ ]:
def build_discriminator():
    model = Sequential()
    model.add(Conv2D(32, kernel_size=3, strides=2, input_shape=(64, 64, 3), padding="same"))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.3))
    model.add(Conv2D(64, kernel_size=3, strides=2, padding="same"))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.3))
    model.add(Conv2D(128, kernel_size=3, strides=2, padding="same"))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.3))
    model.add(Flatten())
    model.add(Dense(1, activation='sigmoid'))
    return model

generator = build_generator()
discriminator = build_discriminator()


Compile GAN

Compile the GAN with an optimizer and loss function.

In [ ]:
from tensorflow.keras.optimizers import Adam

def compile_gan(generator, discriminator):
    discriminator.compile(loss='binary_crossentropy', optimizer=Adam(learning_rate=0.0002, beta_1=0.5), metrics=['accuracy'])
    discriminator.trainable = False
    gan_input = tf.keras.Input(shape=(100,))
    x = generator(gan_input)
    gan_output = discriminator(x)
    gan = tf.keras.Model(gan_input, gan_output)
    gan.compile(loss='binary_crossentropy', optimizer=Adam(learning_rate=0.0002, beta_1=0.5))
    return gan

gan = compile_gan(generator, discriminator)


Optimized Training Process
Training with Data Augmentation and Regularization

In [ ]:
def train_gan(generator, discriminator, gan, epochs, batch_size):
    real_images = np.array(load_images_from_folder('path_to_real_images'))
    fake_images = np.array(load_images_from_folder('path_to_fake_images'))

    for epoch in range(epochs):
        noise = np.random.normal(0, 1, (batch_size, 100))
        generated_images = generator.predict(noise)

        real_images_batch = real_images[np.random.randint(0, real_images.shape[0], batch_size)]
        fake_images_batch = generated_images

        combined_images = np.concatenate([real_images_batch, fake_images_batch])
        labels = np.concatenate([np.ones((batch_size, 1)), np.zeros((batch_size, 1))])
        labels += 0.05 * np.random.random(labels.shape)

        d_loss = discriminator.train_on_batch(combined_images, labels)

        noise = np.random.normal(0, 1, (batch_size, 100))
        misleading_labels = np.ones((batch_size, 1))
        g_loss = gan.train_on_batch(noise, misleading_labels)

        print(f"{epoch} [D loss: {d_loss[0]}] [D accuracy: {d_loss[1]}] [G loss: {g_loss}]")

train_gan(generator, discriminator, gan, epochs=10000, batch_size=64)


Evaluation and Fine-Tuning

In [ ]:
from sklearn.metrics import roc_auc_score

def evaluate_model(generator, discriminator):
    noise = np.random.normal(0, 1, (len(real_images), 100))
    generated_images = generator.predict(noise)

    real_labels = np.ones((len(real_images), 1))
    fake_labels = np.zeros((len(fake_images), 1))

    real_predictions = discriminator.predict(np.array(real_images))
    fake_predictions = discriminator.predict(generated_images)

    y_true = np.concatenate([real_labels, fake_labels])
    y_pred = np.concatenate([real_predictions, fake_predictions])

    accuracy = accuracy_score(y_true, y_pred > 0.5)
    precision = precision_score(y_true, y_pred > 0.5)
    recall = recall_score(y_true, y_pred > 0.5)
    f1 = f1_score(y_true, y_pred > 0.5)
    roc_auc = roc_auc_score(y_true, y_pred)

    print(f"Accuracy: {accuracy}, Precision: {precision}, Recall: {recall}, F1-Score: {f1}, ROC-AUC: {roc_auc}")

evaluate_model(generator, discriminator)
